In [40]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout

In [22]:
import pandas as pd

df = pd.read_excel('Bike Sales Dashboard.xlsx', sheet_name='bike_buyers')
df.head()

,ID,Martial Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Age Brackets,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,Middle Age,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,Middle Age,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,Old,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Middle Age,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Middle Age,Yes


In [23]:
df.nunique()

ID                  1000
Martial Status         2
Gender                 2
Income                16
Children               6
Education              5
Occupation             5
Home Owner             2
Cars                   5
Commute Distance       5
Region                 3
Age                   53
Age Brackets           3
Purchased Bike         2
dtype: int64

In [24]:
##unique values of each feature
columns=["Martial Status","Gender","Children","Education","Occupation","Home Owner","Cars","Commute Distance","Age Brackets","Region","Purchased Bike"]

for col in columns:
    print(col, df[col].unique())

Martial Status ['Married' 'Single']
Gender ['Female' 'Male']
Children [1 3 5 0 2 4]
Education ['Bachelors' 'Partial College' 'High School' 'Partial High School'
 'Graduate Degree']
Occupation ['Skilled Manual' 'Clerical' 'Professional' 'Manual' 'Management']
Home Owner ['Yes' 'No']
Cars [0 1 2 4 3]
Commute Distance ['0-1 Miles' '2-5 Miles' '5-10 Miles' '1-2 Miles' 'More than 10 Miles']
Age Brackets ['Middle Age' 'Old' 'Adolescent']
Region ['Europe' 'Pacific' 'North America']
Purchased Bike ['No' 'Yes']


In [25]:
df.isna().sum()

ID                  0
Martial Status      0
Gender              0
Income              0
Children            0
Education           0
Occupation          0
Home Owner          0
Cars                0
Commute Distance    0
Region              0
Age                 0
Age Brackets        0
Purchased Bike      0
dtype: int64

In [26]:
#mapping ordinal values to numerical values
df['Occupation'] = df['Occupation'].map({'Clerical': 0, 'Manual': 1,'Skilled Manual': 2,'Management': 3,'Professional': 4})
df['Education'] = df['Education'].map({'Partial High School': 0, 'High School': 1, 'Partial College': 2, 'Bachelors': 3, 'Graduate Degree': 4})
df['Commute Distance'] = df['Commute Distance'].map({'0-1 Miles': 0, '1-2 Miles': 1, '2-5 Miles': 2, '5-10 Miles': 3, 'More than 10 Miles': 4})

#label encoding
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['Martial Status'] = labelencoder.fit_transform(df['Martial Status'])
df['Gender']= labelencoder.fit_transform(df['Gender'])
df['Home Owner'] = labelencoder.fit_transform(df['Home Owner'])
df['Region'] = labelencoder.fit_transform(df['Region'])
df['Purchased Bike'] = labelencoder.fit_transform(df['Purchased Bike']) 


In [27]:
df.head()

,ID,Martial Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Age Brackets,Purchased Bike
0,12496,0,0,40000,1,3,2,1,0,0,0,42,Middle Age,0
1,24107,0,1,30000,3,2,0,1,1,0,0,43,Middle Age,0
2,14177,0,1,80000,5,2,4,0,2,2,0,60,Old,0
3,24381,1,1,70000,0,3,4,1,1,3,2,41,Middle Age,1
4,25597,1,1,30000,0,3,0,0,0,0,0,36,Middle Age,1


In [28]:
df = df.drop(['ID','Age Brackets'], axis=1)

In [29]:
from sklearn.model_selection import train_test_split
X = df.drop('Purchased Bike', axis=1)
y = df['Purchased Bike']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
model = Sequential([
    Dense(256, activation='tanh', input_shape=(11,)),  # Input layer
    Dense(128, activation='relu'),                            # Hidden layer 1
    Dropout(0.1),                                            # Dropout layer 
    Dense(64, activation='relu'),                            # Hidden layer 2
    Dense(1, activation='sigmoid')                           # Output layer
])

c:\Users\rodea\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [49]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                │ (None, 256)            │         3,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,289 (173.00 KB)

 Trainable params: 44,289 (173.00 KB)

 Non-trainable params: 0 (0.00 B)

In [50]:
model.fit(X_train, y_train, epochs=200, batch_size=16, verbose=1)

Epoch 1/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4980 - loss: 0.6940 
Epoch 2/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5170 - loss: 0.6927
Epoch 3/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5273 - loss: 0.6920
Epoch 4/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5300 - loss: 0.6920
Epoch 5/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5315 - loss: 0.6919
Epoch 6/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5158 - loss: 0.6928
Epoch 7/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5177 - loss: 0.6926
Epoch 8/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5259 - loss: 0.6921
Epoch 9/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step - accuracy: 0.5265 - loss: 0.6921
Epoch 10/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step - accuracy: 0.4851 - loss: 0.6946
Epoch 11/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step - accuracy: 0.5230 - loss: 0.6923
Epoch 12/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - 

In [51]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 52.999997
